In [1]:
import requests

from lxml import html

import lxml.etree as etree

In [40]:
error_recipes = {}
def get_recipe_details(recipe_url,cuisine,category,sub_category):
    recipe = {}
    response = requests.get(recipe_url)
    doc = html.fromstring(response.content)
    try:
        recipe_name = doc.xpath('//h1[@id = "article-heading_1-0"]')[0].text.strip()
        recipe["name"] = recipe_name
    except:
        return # No recipe name , return
    
    try:
        rating = float(doc.xpath('//div[@id="mntl-recipe-review-bar__rating_1-0"]')[0].text.strip())
    except:
        rating = 0

    recipe["rating"] = rating
    try:
        tagline = doc.xpath('//p[@id="article-subheading_1-0"]')[0].text.strip()
    except:
        tagline = ''

    recipe["tagline"] = tagline
    
    prep_time , cook_time , total_time , servings = 0, 0 , 0 , 1 #default values

    try:
        cooking_times = doc.xpath('//div[@class="mntl-recipe-details__value"]')
        prep_time = cooking_times[0].text.strip()
        
        cook_time = cooking_times[1].text.strip()
        
        total_time = cooking_times[2].text.strip()
        
        servings = cooking_times[3].text.strip()
        
    except:
        pass #Assign default values

    finally:
        recipe["prep_time"] = prep_time
        recipe["cook_time"] = cook_time
        recipe["total_time"] = total_time
        recipe["servings"] = servings

    try:
        ingredients_details = doc.xpath('//ul[@class="mntl-structured-ingredients__list"]')[0].xpath('li')
        ingredients = []
        for ingredient in ingredients_details:
            try:
                spans = ingredient.xpath('p')[0].xpath('span')
                qty = spans[0].text.strip()
                if len(spans) > 2:
                    unit = spans[1].text.strip()
                    name = spans[2].text.strip()
                else:
                    name = spans[1].text.strip()
                split = name.split(', ')
                if len(split) > 1:
                    name,method = split[0],split[1]
                else:
                    method = ''

                ingredients.append((qty,unit,name,method))
            except:
                error_recipes.get(recipe_name,[]).append(''.join(ingredient.itertext()))
            #print(url,''.join(ingredient.itertext()))
    except:
        ingredients = []  #This recipe has no ingredients 
    
    try:
        instructions = []
        steps = doc.xpath('//div[@id="recipe__steps-content_1-0"]')[0].xpath('ol')[0].xpath('li')
        for step in steps:
            instructions.append(step.xpath('p')[0].text.strip())
    except:
        instructions = []
    
    try:
        images = [doc.xpath('//img')[1].attrib['src'].strip()]
        
        for img in doc.xpath('//img')[2:6]:
            src = img.attrib['data-src'].strip()
            images.append(src[:src.find('&w=160')])
    
    except:
        images = []

    recipe['cuisine'] = cuisine
    recipe['category'] = category
    recipe['sub_category'] = sub_category
    
    return recipe,ingredients,instructions,images


In [42]:
def get_recipes(url):
    cuisine = url.split('/')[-4]
    category = url.split('/')[-3]
    sub_category = url.split('/')[-2]
    response = requests.get(url)
    doc = html.fromstring(response.content)
    recipe_cards = doc.xpath('//a[@class="comp mntl-card-list-items mntl-document-card mntl-card card card--no-image"]')
    recipes = []
    ingredients = {}
    instructions = {}
    images = {}
    for recipe_url in recipe_cards[:-1]:
        url = recipe_url.attrib['href']
        recipe,ingredient,steps,imgs = get_recipe_details(url,cuisine,category,sub_category)
        recipes.append(recipe)
        ingredients[recipe["name"]] = ingredient
        instructions[recipe["name"]] = steps
        images[recipe["name"]] = imgs
    return recipes,ingredients,instructions,images
    

In [44]:
recipes = []
ingredients = {}
instructions = {}
images = {}
links = [
    'https://www.allrecipes.com/recipes/15973/world-cuisine/asian/indian/main-dishes/rice/',
    'https://www.allrecipes.com/recipes/1873/world-cuisine/asian/indian/main-dishes/curry/',
    'https://www.allrecipes.com/recipes/1875/world-cuisine/asian/indian/main-dishes/vegetarian/',
    'https://www.allrecipes.com/recipes/1878/world-cuisine/asian/indian/main-dishes/chicken/',
    'https://www.allrecipes.com/recipes/1876/world-cuisine/asian/indian/bread/',
    'https://www.allrecipes.com/recipes/1879/world-cuisine/asian/indian/desserts/',
    'https://www.allrecipes.com/recipes/1877/world-cuisine/asian/indian/side-dishes/',
    'https://www.allrecipes.com/recipes/15935/world-cuisine/asian/indian/drinks/',
    'https://www.allrecipes.com/recipes/17491/world-cuisine/asian/japanese/main-dishes/',
    'https://www.allrecipes.com/recipes/17492/world-cuisine/asian/japanese/soups-and-stews/',
    'https://www.allrecipes.com/recipes/17490/world-cuisine/asian/japanese/appetizers/',
    'https://www.allrecipes.com/recipes/1360/world-cuisine/european/italian/main-dishes/pork/',
    'https://www.allrecipes.com/recipes/1366/world-cuisine/european/italian/main-dishes/turkey/',
    'https://www.allrecipes.com/recipes/1794/world-cuisine/european/italian/main-dishes/beef/',
    'https://www.allrecipes.com/recipes/1796/world-cuisine/european/italian/main-dishes/chicken/',
    'https://www.allrecipes.com/recipes/1797/world-cuisine/european/italian/main-dishes/seafood/',
    'https://www.allrecipes.com/recipes/502/main-dish/pasta/lasagna/',
    'https://www.allrecipes.com/recipes/17551/world-cuisine/european/italian/drinks/',
    'https://allrecipes.com/recipes/1789/world-cuisine/european/italian/authentic/',
    'https://www.allrecipes.com/recipes/1790/world-cuisine/european/italian/soups-and-stews/',
    'https://www.allrecipes.com/recipes/1791/world-cuisine/european/italian/desserts/',
    'https://www.allrecipes.com/recipes/1792/world-cuisine/european/italian/side-dishes/',
    'https://www.allrecipes.com/recipes/1793/world-cuisine/european/italian/appetizers/',
    'https://www.allrecipes.com/recipes/1798/world-cuisine/european/italian/bread/',
    'https://www.allrecipes.com/recipes/1800/world-cuisine/european/italian/salads/',
    'https://www.allrecipes.com/recipes/250/main-dish/pizza/',


]

for link in links:
    r,i,s,im = get_recipes(link)
    recipes += r
    ingredients.update(i)
    instructions.update(s)
    images.update(im)
    print(link+" done ")




https://www.allrecipes.com/recipes/15973/world-cuisine/asian/indian/main-dishes/rice/ done 
https://www.allrecipes.com/recipes/1873/world-cuisine/asian/indian/main-dishes/curry/ done 
https://www.allrecipes.com/recipes/1875/world-cuisine/asian/indian/main-dishes/vegetarian/ done 
https://www.allrecipes.com/recipes/1878/world-cuisine/asian/indian/main-dishes/chicken/ done 
https://www.allrecipes.com/recipes/1876/world-cuisine/asian/indian/bread/ done 
https://www.allrecipes.com/recipes/1879/world-cuisine/asian/indian/desserts/ done 
https://www.allrecipes.com/recipes/1877/world-cuisine/asian/indian/side-dishes/ done 
https://www.allrecipes.com/recipes/15935/world-cuisine/asian/indian/drinks/ done 
https://www.allrecipes.com/recipes/17491/world-cuisine/asian/japanese/main-dishes/ done 
https://www.allrecipes.com/recipes/17492/world-cuisine/asian/japanese/soups-and-stews/ done 
https://www.allrecipes.com/recipes/17490/world-cuisine/asian/japanese/appetizers/ done 
https://www.allrecipes.c

In [45]:
import csv
def recipes_to_csv(recipes: list , recipe_filename:str):
    with open(recipe_filename,'w') as f:
        writer = csv.writer(f)
        writer.writerow(recipes[0].keys())
        
        for recipe in recipes:
            # Convert each value in the recipe dictionary to a list
            recipe_values = list(recipe.values())
            writer.writerow(recipe_values)


recipes_to_csv(recipes,'recipes.csv')


In [46]:
def ingredients_to_csv(ingredients: dict , ingredients_filename:str):
    with open(ingredients_filename,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('recipe','qty','unit','name','method'))
        
        for recipe in ingredients:
            for ing in ingredients[recipe]:
                writer.writerow((recipe,)+ing)


ingredients_to_csv(ingredients,'ingredients.csv')

In [47]:
def instructions_to_csv(instructions: dict , instructions_filename:str):
    with open(instructions_filename,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('recipe','step','instruction'))
        
        for recipe in instructions:
            for step_no,step in enumerate(instructions[recipe]):
                writer.writerow((recipe,)+(step_no+1,step))


instructions_to_csv(instructions,'instructions.csv')

In [48]:
def images_to_csv(images: dict , images_filename:str):
    with open(images_filename,'w') as f:
        writer = csv.writer(f)
        writer.writerow(('recipe','image'))
        for recipe in images:
            for img in enumerate(images[recipe]):
                writer.writerow((recipe,)+(img))


images_to_csv(images,'images.csv')